In [2]:
from pyedflib import highlevel
import pyedflib as plib
import numpy as np
import matplotlib.pyplot as plt

def read_file_signal(file_path, index):
    f = plib.EdfReader(file_path)
    signal = f.readSignal(index)
    f.close()
    return signal

def signal_labels(file_path):
    f = plib.EdfReader(file_path)
    labels = f.getSignalLabels()
    f.close()
    return labels

def combined_signal(file_list, index):
    combined_signals = []
    for file in file_list:
        temp = read_file_signal(file, index)
        combined_signals.append(temp)
    return np.concatenate(combined_signals)

edf_files_severe = [
    "c:\\Users\\anany\\Downloads\\00000066-297469[001].edf",
    "c:\\Users\\anany\\Downloads\\00000066-297469[002].edf",
    "c:\\Users\\anany\\Downloads\\00000066-297469[003].edf",
    "c:\\Users\\anany\\Downloads\\00000066-297469[004].edf",
    "c:\\Users\\anany\\Downloads\\00000066-297469[005].edf",
    "c:\\Users\\anany\\Downloads\\00000066-297469[006].edf",
    "c:\\Users\\anany\\Downloads\\00000066-297469[007].edf"
]

edf_files_normal = [
    "C:\\Users\\anany\\Downloads\\00000064-297469[001].edf",
    "C:\\Users\\anany\\Downloads\\00000064-297469[002].edf",
    "C:\\Users\\anany\\Downloads\\00000064-297469[003].edf",
    "C:\\Users\\anany\\Downloads\\00000064-297469[004].edf",
    "C:\\Users\\anany\\Downloads\\00000064-297469[005].edf",
    "C:\\Users\\anany\\Downloads\\00000064-297469[006].edf",
    "C:\\Users\\anany\\Downloads\\00000064-297469[007].edf",
    "C:\\Users\\anany\\Downloads\\00000064-297469[008].edf",
    "C:\\Users\\anany\\Downloads\\00000064-297469[009].edf"
]

edf_files_moderate = [
    "C:\\Users\\anany\\Downloads\\00000074-297469[001].edf",
    "C:\\Users\\anany\\Downloads\\00000074-297469[002].edf",
    "C:\\Users\\anany\\Downloads\\00000074-297469[003].edf",
    "C:\\Users\\anany\\Downloads\\00000074-297469[004].edf",
    "C:\\Users\\anany\\Downloads\\00000074-297469[005].edf",
    "C:\\Users\\anany\\Downloads\\00000074-297469[006].edf",
    "C:\\Users\\anany\\Downloads\\00000074-297469[007].edf",
    "C:\\Users\\anany\\Downloads\\00000074-297469[008].edf",
    "C:\\Users\\anany\\Downloads\\00000074-297469[009].edf"
]

edf_files_mild = [
    "C:\\Users\\anany\\Downloads\\00000067-297469[001].edf",
    "C:\\Users\\anany\\Downloads\\00000067-297469[002].edf",
    "C:\\Users\\anany\\Downloads\\00000067-297469[003].edf",
    "C:\\Users\\anany\\Downloads\\00000067-297469[004].edf",
    "C:\\Users\\anany\\Downloads\\00000067-297469[005].edf",
    "C:\\Users\\anany\\Downloads\\00000067-297469[006].edf",
    "C:\\Users\\anany\\Downloads\\00000067-297469[007].edf",
    "C:\\Users\\anany\\Downloads\\00000067-297469[008].edf",
    "C:\\Users\\anany\\Downloads\\00000067-297469[009].edf",
    "C:\\Users\\anany\\Downloads\\00000067-297469[010].edf",
    "C:\\Users\\anany\\Downloads\\00000067-297469[011].edf",
]

labels = signal_labels(edf_files_normal[0])
print(labels)
labels[16] = "Flow patient 1"
labels[17] = "Flow patient 2"
labels[23] = "Flow patient 3"
cv_dict = {}

['EEG F3-A2', 'EEG F4-A1', 'EEG A1-A2', 'EEG C3-A2', 'EEG C4-A1', 'EEG O1-A2', 'EEG O2-A1', 'EOG LOC-A2', 'EOG ROC-A2', 'EMG Chin', 'Leg 1', 'Leg 2', 'ECG I', 'RR', 'ECG II', 'Snore', 'Flow Patient', 'Flow Patient', 'Effort THO', 'Effort ABD', 'SpO2', 'Pleth', 'Body', 'Flow Patient', 'xPAP CPAP', 'xPAP IPAP', 'xPAP EPAP', 'Leak Total', 'PulseRate', 'PressCheck', 'ECG IIHF', 'Technical']


In [3]:
import statistics as stat
import matplotlib.pyplot as plt

def plot_signal(label, signal):
    plt.figure(figsize=(20, 10))
    plt.plot(signal)
    plt.title(f"{label} signal")
    plt.xlabel("time")
    plt.ylabel("amplitude")
    # plt.ylim(-300, 300)
    plt.show()

def thresholding(signal, threshold, replace_with):
    mean = signal.mean()
    std = stat.stdev(signal)
    lower_limit = mean - threshold * std
    upper_limit = mean + threshold * std
    filtered_signal = np.where((signal >= lower_limit) & (signal <= upper_limit), signal, replace_with)
    return filtered_signal

def resample_signal(signal, num_samples):
    original_indices = np.linspace(0, len(signal) - 1, num=len(signal))
    resampled_indices = np.linspace(0, len(signal) - 1, num=num_samples)
    resampled_signal = np.interp(resampled_indices, original_indices, signal)
    return resampled_signal

In [4]:
normal_dnd_signals = []

for i in range(32):
    combined_sig = combined_signal(edf_files_normal, i)
    resampled_signal = resample_signal(combined_sig, 25200)
    if i in [23, 37]:
        thresholded_signal = thresholding(resampled_signal, 2, 0)
    else:
        thresholded_signal = thresholding(resampled_signal, 2, resampled_signal.mean())
    normal_dnd_signals.append(thresholded_signal)

In [5]:
mild_dnd_signals = []

for i in range(32):
    combined_sig = combined_signal(edf_files_mild, i)
    resampled_signal = resample_signal(combined_sig, 25200)
    thresholded_signal = thresholding(resampled_signal, 2, resampled_signal.mean())
    mild_dnd_signals.append(thresholded_signal)

In [6]:
moderate_dnd_signals = []

for i in range(32):
    combined_sig = combined_signal(edf_files_moderate, i)
    resampled_signal = resample_signal(combined_sig, 25200)
    thresholded_signal = thresholding(resampled_signal, 2, resampled_signal.mean())
    moderate_dnd_signals.append(thresholded_signal)

In [7]:
severe_dnd_signals = []

for i in range(32):
    combined_sig = combined_signal(edf_files_severe, i)
    resampled_signal = resample_signal(combined_sig, 25200)
    thresholded_signal = thresholding(resampled_signal, 2, resampled_signal.mean())
    severe_dnd_signals.append(thresholded_signal)

In [8]:
import statistics as stats
def find_cv(channel):
    mean = channel.mean()
    var = stats.variance(channel)
    if mean == 0:
        return 'nil'
    return var/mean

In [9]:
combined_cv_dict = {}

for i in range(32):
    if i in [22, 29, 31]:
        continue
    combined_cv_dict[labels[i]] = []
    cv_1 = find_cv(normal_dnd_signals[i])
    cv_2 = find_cv(mild_dnd_signals[i])
    cv_3 = find_cv(moderate_dnd_signals[i])
    cv_4 = find_cv(severe_dnd_signals[i])
    combined_cv_dict[labels[i]].append(cv_1)
    combined_cv_dict[labels[i]].append(cv_2)
    combined_cv_dict[labels[i]].append(cv_3)
    combined_cv_dict[labels[i]].append(cv_4)

print(combined_cv_dict)

{'EEG F3-A2': [-213.80383869963416, -80.52653796638246, -42.180295603685444, -68.99165807749296], 'EEG F4-A1': [-43.461243962070974, -44.15736849196938, -17.036283535533382, -27.00533467139078], 'EEG A1-A2': [-12.424337294233709, -117.52137845892541, 7.2985204826246, 15.575334150965125], 'EEG C3-A2': [-84.13530105007662, -39.210157439129254, -19.81696387756552, -40.92276616103842], 'EEG C4-A1': [53.295106558120374, 29.802882366473295, 17.6337690524196, 38.33348128728897], 'EEG O1-A2': [2.1638240431878226, 0.2715491834519041, 1.5120125020584299, 0.3608391566443004], 'EEG O2-A1': [1.4904117386164446, 0.14705732190331894, 0.8716571482391934, 0.15095465748563194], 'EOG LOC-A2': [46.23739983925256, 23.79556887631528, 14.098735307422421, 33.09692798596699], 'EOG ROC-A2': [-121.49384919160944, -81.52895782363076, -42.819183527898986, -89.60871654405291], 'EMG Chin': [-180.77500074942654, -99.11459226144761, -130.0918583144745, -366.2999435460866], 'Leg 1': [-65.64848645528355, -149.4330652414

In [10]:
channel_variability = {}

for channel, vmr_values in combined_cv_dict.items():
    # abs_vmr_values = [abs(value) for value in vmr_values]
    # variability = max(abs_vmr_values) - min(abs_vmr_values)
    # channel_variability[channel] = variability
    abs_vmr_values = []
    for value in vmr_values:
        try:
            abs_vmr_values.append(abs(float(value)))
        except ValueError:
            continue  # Skip non-numeric values
    if abs_vmr_values:  # Ensure the list is not empty
        variability = max(abs_vmr_values) - min(abs_vmr_values)
        channel_variability[channel] = variability

sorted_channels = sorted(channel_variability, key=channel_variability.get, reverse=True)

top_10_channels = sorted_channels[:10]
print("Top 10 channels based on variability:")
for channel in sorted_channels:
    print(f"{channel}: {channel_variability[channel]}")

Top 10 channels based on variability:
ECG II: 2516.707202369467
ECG IIHF: 2325.5203800319114
Pleth: 1949.7916498130055
Flow patient 1: 1306.215918392189
ECG I: 1139.6612116393183
Flow patient 2: 362.8949145505953
Snore: 343.4504521728328
Flow patient 3: 281.1518655252803
Effort THO: 280.25710525730085
EMG Chin: 267.18535128463895
Effort ABD: 179.32740232912204
EEG F3-A2: 171.62354309594872
Leg 2: 115.71429246206594
EEG A1-A2: 110.22285797630082
Leg 1: 94.64253476255857
EOG ROC-A2: 78.67466566371046
EEG C3-A2: 64.3183371725111
EEG C4-A1: 35.66133750570077
Leak Total: 34.023725168511746
EOG LOC-A2: 32.13866453183014
EEG F4-A1: 27.121084956435997
xPAP CPAP: 6.930288717391482
xPAP IPAP: 6.930288717391482
xPAP EPAP: 3.1302467379558947
EEG O1-A2: 1.8922748597359185
EEG O2-A1: 1.3433544167131257
RR: 0.7466902785314621
PulseRate: 0.6376619931231701
SpO2: 0.3725922041744621


In [11]:
print(channel_variability)

{'EEG F3-A2': 171.62354309594872, 'EEG F4-A1': 27.121084956435997, 'EEG A1-A2': 110.22285797630082, 'EEG C3-A2': 64.3183371725111, 'EEG C4-A1': 35.66133750570077, 'EEG O1-A2': 1.8922748597359185, 'EEG O2-A1': 1.3433544167131257, 'EOG LOC-A2': 32.13866453183014, 'EOG ROC-A2': 78.67466566371046, 'EMG Chin': 267.18535128463895, 'Leg 1': 94.64253476255857, 'Leg 2': 115.71429246206594, 'ECG I': 1139.6612116393183, 'RR': 0.7466902785314621, 'ECG II': 2516.707202369467, 'Snore': 343.4504521728328, 'Flow patient 1': 1306.215918392189, 'Flow patient 2': 362.8949145505953, 'Effort THO': 280.25710525730085, 'Effort ABD': 179.32740232912204, 'SpO2': 0.3725922041744621, 'Pleth': 1949.7916498130055, 'Flow patient 3': 281.1518655252803, 'xPAP CPAP': 6.930288717391482, 'xPAP IPAP': 6.930288717391482, 'xPAP EPAP': 3.1302467379558947, 'Leak Total': 34.023725168511746, 'PulseRate': 0.6376619931231701, 'ECG IIHF': 2325.5203800319114}
